In [22]:
from urllib.request import urlopen

DELIM_BEGIN = "<!-- mode=normal -->"
DELIM_END = "<!-- NEWIMAGE2 -->"

In [23]:
def get_substring_between_delimiters(s: str, start: str, end: str):
    start_index = s.find(start)
    if start_index == -1:
        return None
    start_index += len(start)
    end_index = s.find(end, start_index)
    if end_index == -1:
        return None

    return s[start_index: end_index]

def clean(s: str, tag: str):
    return s.replace(tag, '')

# def scrape_volume(url: str):
    

def scrape_page(url: str) -> str:
    page = urlopen(url)
    html = page.read().decode("utf-8")
    html = get_substring_between_delimiters(html, DELIM_BEGIN, DELIM_END)
    cleaned = clean(html, "<br>")
    return cleaned

def create_url(partial_url: str, i: int):
    return partial_url + f"{i:04d}" + ".html"

In [27]:

uggla_volym_1 = "https://runeberg.org/nfba/"



uggla_url_range_b = "abcdefghijklmnopqrst"
uggla_url_range_c = "abcdefghijklmn"

#supplement?

url_last = "https://runeberg.org/nfba/0824.html"

volume_pages: list[str] = []

volume_start_number = 13
volume_end_number = 824
i = volume_start_number
while(True):
    url = create_url(uggla_volym_1, i)
    volume_pages.append(scrape_page(url))
    print(volume_pages[i - volume_start_number][:10])
    i += 1
    if i > volume_end_number:
        break


#hur vet vi att det är ett uppslagsord?

#uppgeupplaga: fetstilt (<b>ord</b>)
#fjärde: newlines

NORDISK FA
<i>djäfvul
den liflig
för stifts
amtman i B
stora råde
1859 öppna
<b>Abadie<
brukades a
<b>d&rsquo
en kort, s

Näst männ
<b>Abbuna,
motgångar 
till ett s
reformsträ
han där ti
dels i fra
för de ung
Englands r
&quot;Afri
härkomst, 


KeyboardInterrupt: 